In [14]:
import psycopg2
import pandas as pd

In [15]:
#Passando o nome dos arquivos
df_lista = ['dadosPNADc_brutos2019_1.csv', 'dadosPNADc_brutos2019_2.csv']
# Estados que serão filtrados
estados =  ['Rondônia', 'Minas Gerais', 'São Paulo', 'Rio de Janeiro', 'Bahia']

In [16]:
def criar_tabela(cursor, df, nome_tabela):
    mapeamento_tipos = {
        'int64': 'INTEGER',
        'float64': 'FLOAT',
        'object': 'TEXT',
    }
    tipos = df.dtypes.apply(lambda x: mapeamento_tipos.get(str(x), 'TEXT'))
    query = f"CREATE TABLE IF NOT EXISTS {nome_tabela} ({', '.join([f'{col} {tipo}' for col, tipo in tipos.items()])})"
    cursor.execute(query)

In [17]:
def inserir_dados(cursor, df, nome_tabela):
    placeholders = ', '.join(['%s'] * len(df.columns))
    query = f"INSERT INTO {nome_tabela} VALUES ({placeholders});"

    for _, row in df.iterrows():
        cursor.execute(query, tuple(row))

In [18]:
def conecta_db(database, user, password, host):
  con = psycopg2.connect(host=host, 
                         database=database,
                         user=user, 
                         password=password)
  return con

In [19]:
# Realizando conexão com o banco de dados

# db_config = {


#     'database': 'weka-pnad',
#     'user': 'postgres',
#     'password': 'admin',
#     'host': 'localhost'
# }

# conn = conecta_db(**db_config)

# cur = conn.cursor()

In [20]:
def processo_et(df_lista):
    df_tratado = []
    for i in df_lista:
        df = pd.read_csv(i)
        coluna_indice = 'Unnamed: 0'

        if coluna_indice in df.columns:
            df.drop(columns=['Unnamed: 0'], inplace=True)
            print("Coluna deletada")
        else:
            print("Coluna não encontrada")
            

        df['RM_RIDE'] = df['RM_RIDE'].replace(r'\n', ' ', regex=True)
       
        df_tratado.append(df)

    
    return df_tratado
        

In [21]:
# Criando o bando de dados e importando a coluna

# nome_tabela = "dados_pnad_1"

# criar_tabela(cur, df, nome_tabela)

# inserir_dados(cur, df, nome_tabela)

# conn.commit()
# cur.close()

In [22]:
# Preparando o arquivo para o weka
def prepara_df_para_weka(nome_arq, lista, estados):
    for j in nome_arq:
        for i in lista:
            df = i
            df = df.query("UF in @estados")

            df.dropna(axis=1, how='all')

            colunas_inteiro = df.select_dtypes(include=['int64']).columns
            colunas_float = df.select_dtypes(include=['float64']).columns
            colunas_object = df.select_dtypes(include=['object']).columns

            df_aux = df[colunas_object]

            df_exercicio_2_3 = df.drop(columns=colunas_inteiro)
            df_exercicio_2_3 = df_exercicio_2_3.drop(columns=colunas_float)
            df_exercicio_4 = df.drop(columns=colunas_object)


            df_exercicio_2_3.to_csv(f'csvs_tratados\{j}_trat_exercicio_2_3.csv', index=False)
            df_exercicio_4.to_csv(f'csvs_tratados\{j}_trat_exercicio_4.csv', index=False)
        

<string>:21: SyntaxWarning: invalid escape sequence '\{'
<string>:22: SyntaxWarning: invalid escape sequence '\{'
<>:21: SyntaxWarning: invalid escape sequence '\{'
<>:22: SyntaxWarning: invalid escape sequence '\{'
<string>:21: SyntaxWarning: invalid escape sequence '\{'
<string>:22: SyntaxWarning: invalid escape sequence '\{'
<>:21: SyntaxWarning: invalid escape sequence '\{'
<>:22: SyntaxWarning: invalid escape sequence '\{'
C:\Users\leand\AppData\Local\Temp\ipykernel_19768\3783430891.py:21: SyntaxWarning: invalid escape sequence '\{'
  df_exercicio_2_3.to_csv(f'csvs_tratados\{j}_trat_exercicio_2_3.csv', index=False)
C:\Users\leand\AppData\Local\Temp\ipykernel_19768\3783430891.py:22: SyntaxWarning: invalid escape sequence '\{'
  df_exercicio_4.to_csv(f'csvs_tratados\{j}_trat_exercicio_4.csv', index=False)


In [23]:
df_final = proccess_etl(df_lista)

C:\Users\leand\AppData\Local\Temp\ipykernel_19768\273683317.py:4: DtypeWarning: Columns (4,5,35,45,46,56,89,90,98,99,101,102,107,108,118,120,122,129,130,132,133,138,139,143,144,145,146,148,149,151,152,153,154,155,156,158,159,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(i)


Coluna deletada


C:\Users\leand\AppData\Local\Temp\ipykernel_19768\273683317.py:4: DtypeWarning: Columns (4,5,35,45,46,89,90,98,99,101,102,107,108,118,120,122,123,129,130,132,133,138,139,143,144,145,146,148,149,153,154,155,156,158,159,173) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(i)


Coluna deletada


In [24]:
df_cleaner_for_weka(df_lista, df_final, estados)